In [1]:
import requests
import pandas as pd
import os
from env import host, user, password

In [2]:
# def read_csv(url):
#     '''
#     This function reads in the csv from the url
#     '''
#     df = pd.read_csv(url)
#     return df

In [7]:
# Trying to read in csv from folder

pd.read_csv(/fitbit/'2018-04-26_through_2018-05-26.csv')

SyntaxError: invalid syntax (<ipython-input-7-8a4401569e53>, line 3)

In [ ]:
r'Path where the CSV file is stored\File name.csv')

In [ ]:
data_files = '/Users/coreysolitaire/codeup-data-science/time-series-exercises/fitbit'
csv_name = '2018-04-26_through_2018-05-26.csv'

pd.read_csv(f"{data_files}{csv_name}")